<a href="https://colab.research.google.com/github/gyuchangShim/yolo_nas_practice/blob/main/custom_dataset_yolo_nas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install super-gradients==3.5.0
!pip install roboflow

In [ ]:
import cv2
import torch

from IPython.display import clear_output
# super-gradients에서 PPYoloELoss를 받아올 때 오류 발생(loss: ~ )
# 그래서 super-gradients --> paddleslim.nas.loss로 변경
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback
from super_gradients.training import models
from super_gradients.training import Trainer
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train,
    coco_detection_yolo_format_val
)

device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
use_cuda = torch.cuda.is_available()
if use_cuda:
  print(torch.cuda.get_device_name(0))

In [3]:
CHECKPOINT_DIR = 'checkpoints'
trainer = Trainer(experiment_name='Detection_yolonas_practice', ckpt_root_dir=CHECKPOINT_DIR)

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="aIU6ug3LhFpJCcpMteMN")
project = rf.workspace("mjuaiimages").project("mju_ai_week02")
version = project.version(1)
dataset = version.download("yolov5")

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

cuda 캐시 삭제

In [5]:
# dataset_setup
dataset_params = {
    'data_dir': '/content/mju_ai_week02-1',
    'train_images_dir': 'train/images',
    'train_labels_dir': 'train/labels',
    'val_images_dir': 'valid/images',
    'val_labels_dir': 'valid/labels',
    'test_images_dir': 'test/images',
    'test_labels_dir': 'test/labels',
    'classes': ['cat', 'dog']
}

In [6]:
train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
      'batch_size': 16,
      'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
      'batch_size': 16,
      'num_workers': 2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
      'batch_size': 16,
      'num_workers': 2
    }
)

clear_output()

Indexing dataset annotations: 100%|██████████| 4/4 [00:00<00:00, 2439.97it/s]


In [7]:
train_data.dataset.transforms
train_data.dataset.dataset_params['transforms'][1]
train_data.dataset.dataset_params['transforms'][1]['DetectionRandomAffine']['degrees'] = 10.42

In [ ]:
train_data.dataset.plot()

In [ ]:
model = models.get('yolo_nas_s',
                   num_classes=len(dataset_params['classes']),
                   pretrained_weights = "coco")

- 모델 인스턴스화: yolo-nas의 3가지 모델(l, s, m) 모두 사용 가능
-> num_classes 인수 & pretrained_weights 추가 필요

In [9]:
# metric_to_watch: mAP@0.50에서 error
# mAP@0.50 인식 불가 -> why?
train_params = {
    'silent_mode': True,
    "average_best_models": True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    # epoch 횟수는 수정 필요
    "max_epochs": 15,
    "mixed_precision": True,
    # 손실 함수 결정
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_matrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

In [12]:
train_params = { # 모델에 다양한 학습 변수가 포함된 Python dictionary
    "average_best_models": True, # True로 설정하면 학습중에 가장 좋은 모델의 평균을 사용하여 최종 모델을 생성
    "warmup_mode": "linear_epoch_step", # linear_epoch_step으로 설정하면 학습률을 선형적으로 증가시킨다.
    "warmup_initial_lr": 1e-6, # 합습률을 le-6 으로 설정, (0.000001)
    "lr_warmup_epochs": 3, # 학습률을 3에폭동안 증가.
    "initial_lr": 5e-4, # 학습률을 5e-4로 시작한다. (0.0005)
    "lr_mode": "cosine",# 학습률을 코사인 함수로 감소시킨다.
    "cosine_final_lr_ratio": 0.1, # 0.1로 설정하면 최종 학습률의 0.1로 감소시킨다.
    "optimizer": "AdamW",# 모델이 실제 결과와 예측 결과의 차이를 최소화 시키는 알고리즘(즉, 사용하려는 손실함수)을 AdamW로 사용
    "optimizer_params": {"weight_decay": 0.0001}, # 가중치 감소를 0.0001로 설정 (weight decay : 모델의 가중치가 너무 커지는 것을 방지하는 규제 기법)
    "zero_weight_decay_on_bias_and_bn": True, # True로 설정하면 바이어스 및 배치 정규화에 가중치 감소를 적용하지 않는다.
    "ema": True, # True로 설정하면 지수 이동 평균 (EMA)를 사용한다.
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},# EMA 감소율을 0.9로 설정하고 감소 유형을 "threshold"로 설정.
    "max_epochs": 10, # 10으로 설정하면 최대 에폭수를 10으로 설정한다.
    "mixed_precision": True, # True로 설정하면 혼합 정밀도를 사용

    "loss": PPYoloELoss(
        use_static_assigner=False, # 정적 할당자 또는 작업 할당 할당자 사용 여부
        num_classes=len(dataset_params['classes']), # 모델이 감지할 수 있는 클래스 수를 지정
        reg_max=16
    ),
    "valid_metrics_list": [ # DetectionMetrics_050를 사용하여 검증 지표 목록을 설정

        DetectionMetrics_050(
            score_thres=0.1, # 검출에 사용되는 최소 점수를 지정, 해당 변수를 높이면 더 정확한 검출을 얻을 수 있지만 검출되는 양이 더 적어질 수 있다.
            top_k_predictions=300, # 각 이미지에 대해 예측할 최대 개수를 지정
            num_cls=len(dataset_params['classes']), # 모델이 감지할 수 있는 클래스 수를 지정
            normalize_targets=True, # 타겟을 정규화할지 여부를 지정

            post_prediction_callback=PPYoloEPostPredictionCallback( # 검출을 조정하는데 사용되는 콜백 함수

                score_threshold=0.01, # 최종 검출에 사용되는 최소 점수를 지정.
                nms_top_k=1000, # Non-Maximum Suppression(NMS) 알고리즘에서 고려할 상위 k개 박스를 지정, (상위 1000개의 박스를 고려)
                max_predictions=300, # 최종 검출에 사용되는 최대 개수를 지정
                nms_threshold=0.7 #  NMS 알고리즘에서 사용할 IoU(Intersection over Union) 임계값을 지정

            )
        )
    ],

    "metric_to_watch": 'mAP@0.50'
}

In [13]:
trainer.train(model=model,
              training_params=train_params,
              train_loader=train_data,
              valid_loader=val_data)

[2024-03-30 10:17:12] INFO - sg_trainer.py - Starting a new run with `run_id=RUN_20240330_101712_634031`
[2024-03-30 10:17:12] INFO - sg_trainer.py - Checkpoints directory: checkpoints/Detection_yolonas_practice/RUN_20240330_101712_634031
[2024-03-30 10:17:12] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}
Object name `linear_epoch_step` is now deprecated. Please replace it with `LinearEpochLRWarmup`.
initialize_param_groups and update_param_groups usages are deprecated since 3.4.0, will be removed in 3.5.0 and have no effect. 
 Assign different learning rates by passing a mapping of layer name prefixes to lr values through initial_lr training hyperparameter (i.e initial_lr={'backbone': 0.01, 'default':0.1})


The console stream is now moved to checkpoints/Detection_yolonas_practice/RUN_20240330_101712_634031/console_Mar30_10_17_12.txt


[2024-03-30 10:17:17] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               1          (1 available on the machine)
    - Full dataset size:            81         (len(train_set))
    - Batch size per GPU:           16         (batch_size)
    - Batch Accumulate:             1          (batch_accumulate)
    - Total batch size:             16         (num_gpus * batch_size)
    - Effective Batch size:         16         (num_gpus * batch_size * batch_accumulate)
    - Iterations per epoch:         5          (len(train_loader))
    - Gradient updates per epoch:   5          (len(train_loader) / batch_accumulate)
    - Model: YoloNAS_S  (19.02M parameters, 19.02M optimized)
    - Learning Rates and Weight Decays:
      - default: (19.02M parameters). LR: 0.0005 (19.02M parameters) WD: 0.0, (42.13K parameters), WD: 0.0001, (18.98M parameters)

[2024-03-30 10:17:17] INFO - sg_trainer.py - Started training for 1

SUMMARY OF EPOCH 0
├── Train
│   ├── Ppyoloeloss/loss_cls = 2.2874
│   ├── Ppyoloeloss/loss_iou = 0.6423
│   ├── Ppyoloeloss/loss_dfl = 0.8177
│   └── Ppyoloeloss/loss = 3.7473
└── Validation
    ├── Ppyoloeloss/loss_cls = 2.4984
    ├── Ppyoloeloss/loss_iou = 0.5014
    ├── Ppyoloeloss/loss_dfl = 0.74
    ├── Ppyoloeloss/loss = 3.7399
    ├── Precision@0.50 = 0.0
    ├── Recall@0.50 = 0.0
    ├── Map@0.50 = 0.0728
    └── F1@0.50 = 0.0



Validating epoch 1: 100%|██████████| 1/1 [00:00<00:00,  2.10it/s]
[2024-03-30 10:17:36] INFO - base_sg_logger.py - Checkpoint saved in checkpoints/Detection_yolonas_practice/RUN_20240330_101712_634031/ckpt_best.pth
[2024-03-30 10:17:36] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.3995339870452881


...

Validating epoch 8: 100%|██████████| 1/1 [00:00<00:00,  1.85it/s]


SUMMARY OF EPOCH 8
├── Train
│   ├── Ppyoloeloss/loss_cls = 1.2744
│   │   ├── Epoch N-1      = 1.2874 (↘ -0.013)
│   │   └── Best until now = 1.2741 (↗ 0.0004)
│   ├── Ppyoloeloss/loss_iou = 0.6395
│   │   ├── Epoch N-1      = 0.6929 (↘ -0.0535)
│   │   └── Best until now = 0.6246 (↗ 0.0149)
│   ├── Ppyoloeloss/loss_dfl = 0.8637
│   │   ├── Epoch N-1      = 0.8969 (↘ -0.0332)
│   │   └── Best until now = 0.8177 (↗ 0.046)
│   └── Ppyoloeloss/loss = 2.7776
│       ├── Epoch N-1      = 2.8773 (↘ -0.0997)
│       └── Best until now = 2.8189 (↘ -0.0413)
└── Validation
    ├── Ppyoloeloss/loss_cls = 161.1541
    │   ├── Epoch N-1      = 661.6823 (↘ -500.5282)
    │   └── Best until now = 0.9358 (↗ 160.2182)
    ├── Ppyoloeloss/loss_iou = 1.3595
    │   ├── Epoch N-1      = 1.4369 (↘ -0.0775)
    │   └── Best until now = 0.5014 (↗ 0.858)
    ├── Ppyoloeloss/loss_dfl = 4.0239
    │   ├── Epoch N-1      = 5.6098 (↘ -1.5859)
    │   └── Best until now = 0.74   (↗ 3.2839)
    ├── Ppyoloeloss/los

Validating epoch 9: 100%|██████████| 1/1 [00:00<00:00,  1.70it/s]


SUMMARY OF EPOCH 9
├── Train
│   ├── Ppyoloeloss/loss_cls = 1.248
│   │   ├── Epoch N-1      = 1.2744 (↘ -0.0264)
│   │   └── Best until now = 1.2741 (↘ -0.0261)
│   ├── Ppyoloeloss/loss_iou = 0.6058
│   │   ├── Epoch N-1      = 0.6395 (↘ -0.0337)
│   │   └── Best until now = 0.6246 (↘ -0.0188)
│   ├── Ppyoloeloss/loss_dfl = 0.8552
│   │   ├── Epoch N-1      = 0.8637 (↘ -0.0085)
│   │   └── Best until now = 0.8177 (↗ 0.0375)
│   └── Ppyoloeloss/loss = 2.709
│       ├── Epoch N-1      = 2.7776 (↘ -0.0686)
│       └── Best until now = 2.7776 (↘ -0.0686)
└── Validation
    ├── Ppyoloeloss/loss_cls = 18.7049
    │   ├── Epoch N-1      = 161.1541 (↘ -142.4491)
    │   └── Best until now = 0.9358 (↗ 17.7691)
    ├── Ppyoloeloss/loss_iou = 1.24
    │   ├── Epoch N-1      = 1.3595 (↘ -0.1195)
    │   └── Best until now = 0.5014 (↗ 0.7386)
    ├── Ppyoloeloss/loss_dfl = 2.3269
    │   ├── Epoch N-1      = 4.0239 (↘ -1.697)
    │   └── Best until now = 0.74   (↗ 1.5869)
    ├── Ppyoloeloss/loss 

[2024-03-30 10:20:00] INFO - sg_trainer.py - RUNNING ADDITIONAL TEST ON THE AVERAGED MODEL...
Validating epoch 10: 100%|██████████| 1/1 [00:00<00:00,  1.58it/s]


In [15]:
best_model = models.get('yolo_nas_s',
                        num_classes=len(dataset_params['classes']),
                        checkpoint_path="/content/checkpoints/Detection_yolonas_practice/RUN_20240330_101712_634031/ckpt_best.pth")

[2024-03-30 10:21:45] INFO - checkpoint_utils.py - Successfully loaded model weights from /content/checkpoints/Detection_yolonas_practice/RUN_20240330_101712_634031/ckpt_best.pth EMA checkpoint.


In [18]:
trainer.test(model=best_model,
             test_loader=test_data,
             test_metrics_list=DetectionMetrics_050(score_thres=0.1,
                                                    top_k_predictions=300,
                                                    num_cls=len(dataset_params['classes']),
                                                    normalize_targets=True,
                                                    post_prediction_callback=PPYoloEPostPredictionCallback(score_threshold=0.01,
                                                                                                           nms_top_k=1000,
                                                                                                           max_predictions=300,
                                                                                                           nms_threshold=0.7)
                                                    ))

Testing:   0%|          | 0/1 [00:00<?, ?it/s]

{'PPYoloELoss/loss_cls': 3.218451,
 'PPYoloELoss/loss_iou': 1.049529,
 'PPYoloELoss/loss_dfl': 1.2484246,
 'PPYoloELoss/loss': 5.5164046,
 'Precision@0.50': 0.004926108289510012,
 'Recall@0.50': 0.5,
 'mAP@0.50': 0.011166086420416832,
 'F1@0.50': 0.009756097570061684}